In [ ]:
!pip install transformers datasets

In [ ]:
#wikiden url ile tek sayfa çalmaca
import requests
from bs4 import BeautifulSoup
import os

# Data klasörünü oluştur
if not os.path.exists('data'):
    os.makedirs('data')

while True:
    # Kullanıcıdan Wikipedia URL'sini al
    url = input("URL or exit : ")

    # Çıkış komutunu kontrol et
    if url.lower() in ['exit', 'quit']:
        print("Program sonlandırılıyor.")
        break

    # Sayfa içeriğini al
    response = requests.get(url)
    if response.status_code != 200:
        print("URL'den veri alınamadı.")
        continue

    # Sayfa içeriğini BeautifulSoup ile ayrıştır
    soup = BeautifulSoup(response.text, 'html.parser')

    # Sayfa başlığını al
    title = soup.find('h1', {'id': 'firstHeading'}).get_text()

    # Ana içerik kısmını seç (Vikipedi sayfalarındaki ana metin genellikle <p> etiketlerinde bulunur)
    content = soup.find_all('p')

    # Metni birleştir
    text = "\n".join([para.get_text().strip() for para in content])

    # Boş satırları temizle
    cleaned_text = "\n".join(line for line in text.splitlines() if line.strip())

    # Başlık için geçerli bir dosya adı oluştur
    filename = f"{title}.txt".replace('/', '_').replace('\\', '_')

    # Dosyayı data klasörüne kaydet
    filepath = os.path.join('data', filename)
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

    print(f"Metin '{filepath}' dosyasına kaydedildi.")


In [ ]:
#data klasörüne bak data.txt diye şeyap
import re
import os

# Data klasöründeki tüm .txt dosyalarını listele
data_folder = 'data'
text_files = [f for f in os.listdir(data_folder) if f.endswith('.txt')]

# Tüm txt dosyalarını yükle ve içeriklerini birleştir
all_text_data = ""
for file_name in text_files:
    file_path = os.path.join(data_folder, file_name)
    print(f"Şu an {file_name} ile işlem yapıyorum.")
    with open(file_path, 'r', encoding='utf-8') as file:
        all_text_data += file.read() + "\n"

# Köşeli parantez içindeki tüm rakamları sil
cleaned_text = re.sub(r'\[\d+\]', '', all_text_data)

# Boş satırları temizle
cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)

# Temizlenmiş datayı data.txt olarak kaydet
with open("data.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_text.strip())  # Strip, başındaki ve sonundaki boşlukları da temizler

print("Tüm veriler temizlendi ve data.txt dosyasına kaydedildi.")


In [ ]:
# Eğitimi burda hallediyoruz
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Model ve tokenizer'ı yükle
model_name = "gpt2"  # veya daha küçük bir versiyonu: "gpt2-small"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Veri kümesini oluştur
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

# data.txt dosyasını oku ve veri kümesini yükle
train_dataset = load_dataset("data.txt", tokenizer)

# Veri hazırlayıcı ve eğitim argümanlarını ayarla
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./ahraz",  # Modelin kaydedileceği dizin
    overwrite_output_dir=True,
    num_train_epochs=1,  # Eğitim döngüsü sayısı
    per_device_train_batch_size=2,  # Eğitim sırasında her cihazda batch boyutu
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer ile modeli eğit
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

# Modeli kaydet
model.save_pretrained("./ahraz")
tokenizer.save_pretrained("./ahraz")


In [ ]:
#burda konuşmaç
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Model ve tokenizer'ı yükle
tokenizer = GPT2Tokenizer.from_pretrained("./ahraz")
model = GPT2LMHeadModel.from_pretrained("./ahraz")

def chat(input_text):
    # Girişleri encode et
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Attention mask oluştur
    attention_mask = torch.ones(inputs.shape, dtype=torch.long)

    # Modeli çalıştır
    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,  # Attention mask'ı ekle
        max_length=80,      # Yanıtın maksimum uzunluğu
        num_return_sequences=1,  # Döndürülecek yanıt sayısı
        do_sample=True,      # Sampling (örnekleme) modunu aktif et
        temperature=0.7,     # Rastgelelik oranı
        top_k=50,            # En yüksek olasılıklı token'ları dikkate al
        top_p=0.85,          # Nucleus sampling için olasılık eşikleri
        pad_token_id=tokenizer.eos_token_id  # Padding token'ı ayarla
    )

    # Yanıtı decode et
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Soruyu yanıtın başından çıkar
    response = decoded_output[len(input_text):].strip()
    #response = decoded_output

    return response

# Sohbeti başlat
while True:
    user_input = input("you: ")
    if user_input.lower() in ["quit", "exit"]:
        print("ahraz: Görüşürüz!")
        break
    response = chat(user_input)
    print("ahraz:", response)
